In [ ]:
import numpy as np
from keras.utils import np_utils
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Pentru preprocesari
#from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
data = pd.read_csv('train.csv')

# Extract the image file names and labels
image_files = data['Image'].values
labels = data['Class'].values

# Initialize empty lists for train images and train labels
train_images = []
train_labels = []

#image_directory = 'C:/Users/elena/OneDrive/Desktop/FMI AN II SEM II/ML/competitie/train_images'

image_directory = os.path.join(os.getcwd(), 'train_images')

# Loop through each image file and label
for image_file, label in zip(image_files, labels):

    image_path = os.path.join(image_directory, image_file)
    
    # Read and process the image
    image = Image.open(image_path)
    image = image.resize((32, 32))

    
    # Normalize the pixel values to the range [0, 1]
    image = np.array(image) / 255.0
    
    # Append the image and label to the lists
    train_images.append(image)
    train_labels.append(label)

# Convert the lists to NumPy arrays
train_images = np.array(train_images)
#train_images = np.expand_dims(train_images, axis=3)
train_labels = np.array(train_labels)

print(train_images.shape)
train_images, train_labels = shuffle(train_images, train_labels) # amestecare

categorie_train=np_utils.to_categorical(train_labels, 96) 

In [ ]:

data = pd.read_csv('test.csv')

# Extract the image file names

image_files = data['Image'].values

# Initialize an empty list for test images
test_images = []

# Define the image directory
image_directory = os.path.join(os.getcwd(), 'test_images')

# Loop through each image file
for image_file in image_files:

    image_path = os.path.join(image_directory, image_file)
    
    # Read and process the image
    image = Image.open(image_path)
    image = image.resize((32, 32))
    
    # Normalize the pixel values to the range [0, 1]
    image = np.array(image) / 255.0
    
    # Append the image and label to the lists
    test_images.append(image)

# Convert the list to a NumPy array
test_images = np.array(test_images)

# Print the shape of the test images array
print(test_images.shape)


In [ ]:
data = pd.read_csv('val.csv')

# Extract the image file names and labels
v_image_files = data['Image'].values
v_labels = data['Class'].values

# Initialize empty lists for train images and train labels
val_images = []
val_labels = []

#image_directory = 'C:/Users/elena/OneDrive/Desktop/FMI AN II SEM II/ML/competitie/train_images'

image_directory = os.path.join(os.getcwd(), 'val_images')

# Loop through each image file and label
for image_file, label in zip(v_image_files, v_labels):

    image_path = os.path.join(image_directory, image_file)
    
    # Read and process the image
    image = Image.open(image_path)
    image = image.resize((32, 32))
    
    # Normalize the pixel values to the range [0, 1]
    image = np.array(image) / 255.0
    
    # Append the image and label to the lists
    val_images.append(image)
    val_labels.append(label)

# Convert the lists to NumPy arrays
val_images = np.array(val_images)
#train_images = np.expand_dims(train_images, axis=3)
val_labels = np.array(val_labels)

print(val_images.shape)
val_images, val_labels = shuffle(val_images, val_labels) # amestecare


categorie_val=np_utils.to_categorical(val_labels, 96) 

In [ ]:
model = keras.Sequential()

# Add layers to the model
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(96, activation='softmax'))  # 96 classes for multi-way classification


model.summary()

In [ ]:
# configurare model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# antrenare model
history = model.fit(train_images, categorie_train, batch_size=32, epochs=100, validation_data=(val_images, categorie_val))
# evaluare model
loss_accuracy = model.evaluate(val_images, categorie_val)
#test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(loss_accuracy)



In [ ]:
# Perform predictions on the test images
predictions = model.predict(test_images)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

data = pd.read_csv('test.csv')
imageF = data['Image']

# Create a DataFrame for the submissions
# for image in imageF:
#     print(image)

submissions_df = pd.DataFrame({'Image': imageF, 'Class': predicted_labels})

# Save the DataFrame to a submissions.csv file
submissions_df.to_csv('submissions.csv', header=True, index=False)

print("Submission file 'submissions.csv' created successfully.")
